In [2]:
import pandas as pd 

from general_functions import get_file_contents

key = 'zaeHKUd+udWXYAS90iNFVdplnQ58mV0a++SuSZrKRVhjD2cJOhRSDZ9xzMmrxmd5S0TnN8Jg2UZCJLH8Y6oZQg=='
#filename = 'ServiceKey.txt'
#api_key = get_file_contents(filename)

import requests


In [3]:
def load_xml_to_dataframe(response, keyword):
    import bs4 as bs
    import urllib.request
    soup = bs.BeautifulSoup(response_xml,'xml')
    
    rows = soup.find_all(keyword)
    columns = rows[0].find_all()
    
    rowList = []
    nameList = []
    columnList = []

    rowsLen = len(rows)
    columnsLen = len(columns)

    for i in range(0, rowsLen):
        columns = rows[i].find_all()

        for j in range(0, columnsLen):
            if i == 0:
                nameList.append(columns[j].name) #header
            eachColumn = columns[j].text #value
            columnList.append(eachColumn)
        rowList.append(columnList)
        columnList = []    # 다음 row의 값을 넣기 위해 비워준다

    result = pd.DataFrame(rowList, columns=nameList)
    return result

### 계열회사조회


In [4]:
#xml
params ={'serviceKey' : key, 
         'pageNo' : '1', 
         'numOfRows' : '10', 
         'resultType' : 'xml', 
         'basDt' : '20220701' }#, 
         #'crno' : '1101110028939', #계열회사법인등록번호
         #'afilCmpyNm' : '케이비아이알로이(주)' }

url = 'http://apis.data.go.kr/1160100/service/GetCorpBasicInfoService/getAffiliate'
response_xml = requests.get(url, params=params).text.encode('utf-8')


import xmltodict
xmltodict.parse(response_xml)

In [5]:
load_xml_to_dataframe(response_xml, 'item') #.to_excel('계열사회사조회.xlsx')

,afilCmpyCrno,afilCmpyNm,basDt,crno,lstgYn
0,Z,,20220701,0004108280449,N
1,1101110014764,롯데건설(주),20220701,1101110000086,
2,1101110033722,(주)롯데푸드,20220701,1101110000086,
3,1101110076300,롯데지주(주),20220701,1101110000086,
4,1101110145410,호텔롯데(주),20220701,1101110000086,
5,1101110159099,롯데상사(주),20220701,1101110000086,
6,1101110193196,롯데케미칼(주),20220701,1101110000086,
7,1101110578231,롯데글로벌로지스(주),20220701,1101110000086,
8,1101111217416,롯데캐피탈(주),20220701,1101110000086,
9,1101113326588,롯데렌탈(주),20220701,1101110000086,


### 기업개요 조회

In [28]:
url = 'http://apis.data.go.kr/1160100/service/GetCorpBasicInfoService/getCorpOutline'
#params ={'serviceKey' : key, 'pageNo' : '1', 'numOfRows' : '10', 'resultType' : 'xml', 
#         'basDt' : '20200509', 'crno' : '1101113892240', 'corpNm' : '메리츠자산운용' }

params ={'serviceKey' : key, 'pageNo' : '1', 'numOfRows' : '10', 'resultType' : 'xml', 
         'basDt' : '20200509', 'corpNm' : '리만 브라더스' }
#params ={'serviceKey' : key, 'pageNo' : '1', 'numOfRows' : '10', 'resultType' : 'xml', 
#         'basDt' : '20220701'}
response_xml = requests.get(url, params=params).content #.text.encode('utf-8')

load_xml_to_dataframe(response_xml, 'item')['crno'] #.to_excel('기업개요조회.xlsx')

0    0000000000000
Name: crno, dtype: object

### 연결 대상 종속 기업조회

In [18]:
url = 'http://apis.data.go.kr/1160100/service/GetCorpBasicInfoService/getConsSubsComp'
params ={'serviceKey' : key, 'pageNo' : '1', 'numOfRows' : '10', 'resultType' : 'xml', 
         'basDt' : '20220707' }#, 'crno' : '1101110035835', 'sbrdEnpNm' : 'Saudi-Taihan Co. Ltd.' }


response_xml = requests.get(url, params=params).text.encode('utf-8')
#import xmltodict
#xmltodict.parse(response_xml)

In [21]:
load_xml_to_dataframe(response_xml, 'item')#.to_excel('연결 대상 종속 기업 조회.xlsx')

,basDt,crno,dntRltBsisCtt,mainSbrdEnpYnCtt,sbrdEnpEstbDt,sbrdEnpLtstEbzyrTastAmt,sbrdEnpMainBizCtt,sbrdEnpNm,sbrdEnpadr
0,20220707,1101110548771,기업 의결권의 과반수 소유&amp;cr;(기업회계기준서 1027호 13),미해당,20070924,3663035,고온전지&amp;cr;판매,VITZROCELL USA Inc..,10055 Regal Row Ste 180&amp;cr;Houston TX 77040
1,20220707,1101117417458,지분율43.58%,-,20190627,12973242,배터리&amp;cr;소재,(주)에코케미칼,울산광역시 남구 신두왕로 39
2,20220707,1101117417458,지분율100%,-,20150708,274972,ESS&amp;cr;사업,SAT USA LLC,115 STRYKER LANE&amp;cr;HILLSBOROUGH TOWNSHIP ...
3,20220707,1311110558163,-,-,,0,-,-,-
